In [1]:
import pandas as pd
import numpy as np
import os
import PATHS
from PATHS import LOCAL_THESIS as local_thesis_path

In [2]:
def switch_importing_no_formatting(i) -> str:
    if i<10:
        return "00"+str(i)
    if i<100:
        return "0"+str(i)
    else:
        return i
def vehicle_track_path_builder(path:str):
    dataset_path = "test-area-autonomous-driving-dataset-master/datasets/recorded_trackfiles/"
    if "k729" in path:
        return local_thesis_path+dataset_path+"k729_2022-03-16"
    elif "k733_2018" in path:
        return local_thesis_path+dataset_path+"k733_2018-05-02"
    elif "k733_2020" in path:
        return local_thesis_path+dataset_path+"k733_2020-09-15"
    else:
        raise Exception("No correct path provided. Path must contain one of the following:\n k729_2022-03-16 \n k733_2018-05-02 \n k733_2020-09-15") 


In [3]:
def import_func(start:int=0, stop:int=0, path:str=PATHS.FZI_TRACKFILES_K729_2022):
    # local_path = vehicle_track_path_builder(path)
    local_path = path
    # get first file regardless, filter for cars only
    df_accumulated =pd.read_csv(local_path + "/vehicle_tracks_000.csv")
    df_accumulated = df_accumulated.loc[df_accumulated['agent_type'].isin(['Truck', 'Car', 'Bike'])]
    # if needed, add other files
    if stop == 0:
        return df_accumulated
    for i in np.arange(start=start+1, stop=stop+1):
        formatted_import_number = switch_importing_no_formatting(i)
        #print(f"Formatted number: {formatted_import_number}")
        import_string = local_path + "/vehicle_tracks_" + formatted_import_number + ".csv"
        import_df = pd.read_csv(import_string)
        #print(f"importing: {import_string} \n{import_df.head(3)}")
        import_df_filtered_cars = import_df[import_df['agent_type'].isin(['Truck', 'Car', 'Bike'])]
        #print(f"imported: {data_car_only.head}")
        df_accumulated = pd.concat([df_accumulated,import_df_filtered_cars], ignore_index=True)
    return df_accumulated

In [4]:
df_k729 = import_func(0,24,"k729")
df_k733_2018 = import_func(path="k733_2018")
df_k733_2020 = import_func(path="k733_2020")
df_vehicles_accumulated = pd.concat([df_k729,df_k733_2018,df_k733_2020], ignore_index=True)

In [ ]:
# Write acccumulated car data to csv file
try:
    os.remove(local_thesis_path + "vehicle-tracks-accumulated.csv")
except:
    print("no file was removed")
df_vehicles_accumulated.to_csv(path_or_buf=local_thesis_path + "vehicle-tracks-accumulated.csv")